## 1. check corpus

In [ ]:
import torch
import pandas as pd
import numpy as np
# ================= 配置区域 =================
# 请确认这是你用来构建 corpus 的原始 CSV 路径
csv_path = "/Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/abundance_B_13901.csv"
# ===========================================

# --- 1. 基础概览与样本检查 (仅查看最短和最长) ---
pad_id = corpus.tokenizer.pad_token_id
real_lengths = (corpus.tokens != pad_id).sum(dim=1)

targets = [
    ("🟢 最短 (Shortest)", torch.argmin(real_lengths).item()),
    ("🔴 最长 (Longest) ", torch.argmax(real_lengths).item())
]

print(f"📊 [概览] 样本总数: {len(corpus)} | 最短长度: {real_lengths.min()} | 最长长度: {real_lengths.max()}")

for title, idx in targets:
    ids = corpus[idx]['input_ids']
    valid_len = real_lengths[idx].item()
    
    print(f"\n{'='*40}\n🧐 {title} | Index: {idx} | 长度: {valid_len}/{len(ids)}\n{'-'*40}")
    print(f"{'Pos':<5} {'ID':<6} {'Token'}")
    
    # 智能切片：开头5个 ... 中间(EOS附近) ... 结尾
    indices = sorted(list(set(
        list(range(0, 5)) + 
        list(range(valid_len - 2, valid_len + 3)) + 
        list(range(len(ids) - 2, len(ids)))
    )))

    last_i = -1
    for i in indices:
        if i >= len(ids): continue
        if last_i != -1 and i > last_i + 1:
            print(f"{'...':<5} {'...':<6} ...")
        
        token = corpus.tokenizer.convert_ids_to_tokens(ids[i].item())
        print(f"{i:<5} {ids[i]:<6} {token}")
        last_i = i

# --- 2. 词表利用率与丢失诊断 ---
print(f"\n{'='*40}\n🚀 词表利用率诊断\n{'='*40}")

# 统计当前 Corpus 使用情况
unique_ids = set(torch.unique(corpus.tokens).tolist())
all_vocab_ids = set(range(corpus.tokenizer.vocab_size))
special_ids = set(corpus.tokenizer.convert_tokens_to_ids(['<pad>', '<bos>', '<eos>', '<mask>']))
target_vocab_ids = all_vocab_ids - special_ids # 理论上应有的菌属ID

used_vocab = len(unique_ids - special_ids)
missing_ids = target_vocab_ids - unique_ids
missing_genera = [corpus.tokenizer.convert_ids_to_tokens(i) for i in missing_ids]

print(f"💡 词表利用率: {used_vocab}/{len(corpus.tokenizer)} ({used_vocab/len(corpus.tokenizer):.1%})")
print(f"❓ 未出现的菌属数量: {len(missing_genera)}")

# 如果有丢失，读取 CSV 进行比对 (使用修正后的正则)
if len(missing_genera) > 0:
    print(f"\n📖 正在读取 CSV 核对: {csv_path}")
    
    try:
        df_raw = pd.read_csv(csv_path, index_col=0)
        # 关键修正：使用更宽容的正则提取列名，确保能匹配到带特殊字符的菌名
        raw_genera = set(df_raw.columns.str.extract(r'(g__[^;]+)').squeeze().tolist())
        
        not_in_csv = [g for g in missing_genera if g not in raw_genera]
        in_csv_but_zero = [g for g in missing_genera if g in raw_genera]
        
        if not_in_csv:
            print(f"\n❌ [严重] {len(not_in_csv)} 个菌在 CSV 表头中完全找不到:")
            print(f"   (如果数量很多，请检查 corpus.py 中的正则是否已修改为 r'(g__[^;]+)')")
            print(f"   示例: {not_in_csv[:5]} ...")
        
        if in_csv_but_zero:
            print(f"\n📉 [正常] {len(in_csv_but_zero)} 个菌在 CSV 中存在，但在语料库中消失:")
            print(f"   原因: 这些菌在所有样本中的丰度可能均为 0，或者被 Phylogeny 树过滤了。")
            print(f"   示例: {in_csv_but_zero[:5]} ...")
            
    except Exception as e:
        print(f"⚠️ 读取 CSV 失败: {e}")
else:
    print("✅ 完美！所有菌属都已包含 (利用率符合预期)。")

In [ ]:
import pandas as pd
from tqdm import tqdm

# ================= 配置 =================
save_path = project_root / "data" / "try2_withCC/sample_length_stats_B.csv"
# =======================================

print("📊 开始统计每个样本的真实词表长度...")

results = []

# 检查 corpus 是否保留了原始数据
if not hasattr(corpus, 'data') or corpus.data is None:
    print("❌ 错误: corpus 对象中没有找到 .data 属性。")
    print("可能是因为构建时为了省内存执行了 del self.data。")
    print("如果这样，你需要重新加载 CSV 到变量 (例如 df_raw) 并在此处遍历它。")
else:
    # 遍历 corpus 内部存储的原始 dataframe
    # corpus.data 的索引通常就是 Sample ID
    for sample_id in tqdm(corpus.data.index, desc="Analyzing"):
        
        # 1. 获取该样本的一行数据
        row_data = corpus.data.loc[sample_id]
        
        # 2. 调用内部函数计算长度
        # _convert_to_token 返回: (处理后的token列表, 原始长度)
        # 注意：这个 original_len 是包含 <bos> 和 <eos> 的
        _, original_len = corpus._convert_to_token(row_data)
        
        # 3. 计算截断情况
        max_len = corpus.max_len
        # 实际保留的有效长度 (不含 pad)
        kept_len = min(original_len, max_len)
        # 被丢弃的词数量
        truncated_count = max(0, original_len - max_len)
        
        results.append({
            "Sample_ID": sample_id,
            "Original_Length": original_len,   # 原始真实长度
            "Max_Limit": max_len,              # 设定的上限 (512)
            "Truncated_Tokens": truncated_count, # 被截断丢掉的词数
            "Is_Truncated": truncated_count > 0  # 是否发生了截断
        })

    # 4. 生成 DataFrame 并保存
    df_stats = pd.DataFrame(results)
    df_stats.to_csv(save_path, index=False)

    print(f"\n✅ 统计完成！CSV 已保存到: {save_path}")
    
    # 5. 打印一些摘要信息
    trunc_num = df_stats['Is_Truncated'].sum()
    print(f"\n--- 摘要 ---")
    print(f"总样本数: {len(df_stats)}")
    print(f"被截断的样本数: {trunc_num} ({trunc_num/len(df_stats):.2%})")
    if trunc_num > 0:
        max_loss = df_stats['Truncated_Tokens'].max()
        print(f"最严重的截断: 丢掉了 {max_loss} 个词")
        
    # 看看前几行
    print("\n预览生成的表格:")
    print(df_stats.head())